##CGAN on MNIST Dataset

In [ ]:
import tensorflow as tf
import keras
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.models import Model, Sequential
from keras.layers.core import Dense, Dropout
from keras.layers import LeakyReLU
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import initializers
from keras.layers.convolutional import UpSampling2D, Conv2D
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class CGAN():
    def __init__(self):
      # Input Shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10  # we have 10 classes in our dataset, digits from 0-9
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)
        # Learning Rate = 0.0002
        # Momentum Parameter = 0.5


        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()
        # self.generator.compile(loss=['binary_crossentropy'],
        #     optimizer=optimizer,
        #     metrics=['accuracy'])
        # we have Adam optimizer and we use binary_cross_entropy cause an image can either be real or fake

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))  # generating some noise as we want our generator to generate some images
        label = Input(shape=(1,))    ## IMP.. -- labels are also given in CGAN
        img = self.generator([noise, label])
        # This line creates a Keras model object that takes the generator input- [noise,label] as input and outputs the discriminator's classification of the generated images.
        

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image and labels as input and determines validity
        # and the label of that image
        valid = self.discriminator([img, label])

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model([noise, label], valid)
        self.combined.compile(loss=['binary_crossentropy'],
            optimizer=optimizer)
        # This is the combined model of the vanilla GAN, where the generator and discriminator are trained together.
    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim, kernel_initializer = 'uniform', bias_initializer = 'zeros'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(720))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)
        # IMP...
        # we are returning the model with noise and labels together
        # this separates CGAN from Vanilla GAN

    def build_discriminator(self):

        model = Sequential()

        model.add(Dense(1024, input_dim=np.prod(self.img_shape)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        label = Input(shape=(1,), dtype='int32')

        label_embedding = Flatten()(Embedding(self.num_classes, np.prod(self.img_shape))(label))
        flat_img = Flatten()(img)

        model_input = multiply([flat_img, label_embedding])

        validity = model(model_input)

        return Model([img, label], validity)
        # iMP.. Have a look at it
        # we are returning the model with images and labels together

    def train(self, epochs, batch_size=128, sample_interval=50):

        (X_train, y_train), (_, _) = mnist.load_data()

        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs, labels = X_train[idx], y_train[idx]

            noise = np.random.normal(0, 1, (batch_size, 100))

            gen_imgs = self.generator.predict([noise, labels])

            d_loss_real = self.discriminator.train_on_batch([imgs, labels], valid)
            d_loss_fake = self.discriminator.train_on_batch([gen_imgs, labels], fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            sampled_labels = np.random.randint(0, 10, batch_size).reshape(-1, 1)

            g_loss = self.combined.train_on_batch([noise, sampled_labels], valid)

            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 2, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.arange(0, 10).reshape(-1, 1)

        gen_imgs = self.generator.predict([noise, sampled_labels])

        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
                axs[i,j].set_title("Digit: %d" % sampled_labels[cnt])
                axs[i,j].axis('off')
                cnt += 1
        plt.savefig("images%d.png" % epoch)
        plt.close()

In [ ]:
if __name__ == '__main__':
    cgan = CGAN()
    cgan.train(epochs=10000, batch_size=128, sample_interval=200)

Streaming output truncated to the last 5000 lines.
4/4 [==============================] - 0s 11ms/step
7506 [D loss: 0.695899, acc.: 50.78%] [G loss: 0.725031]
4/4 [==============================] - 0s 13ms/step
7507 [D loss: 0.686851, acc.: 54.69%] [G loss: 0.738739]
4/4 [==============================] - 0s 9ms/step
7508 [D loss: 0.688816, acc.: 51.17%] [G loss: 0.733382]
4/4 [==============================] - 0s 10ms/step
7509 [D loss: 0.688141, acc.: 50.39%] [G loss: 0.729911]
4/4 [==============================] - 0s 13ms/step
7510 [D loss: 0.696290, acc.: 50.00%] [G loss: 0.733684]
4/4 [==============================] - 0s 12ms/step
7511 [D loss: 0.680168, acc.: 55.47%] [G loss: 0.721959]
4/4 [==============================] - 0s 10ms/step
7512 [D loss: 0.683513, acc.: 58.20%] [G loss: 0.718264]
4/4 [==============================] - 0s 12ms/step
7513 [D loss: 0.691745, acc.: 53.12%] [G loss: 0.722563]
4/4 [==============================] - 0s 13ms/step
7514 [D loss: 0.683572, ac